In [ ]:
import db

# Drugs and Fake IDs: Data Mining The Dark Web
<hr>

This Jupyter Notebook belongs to a data mining project takign place at UEL. The aim of this research is to monitor a selection of dark web resources to gain insights regarding the type of products and services that are sold, and to possibly identify any significant trends.

On the summer of 2015, a web crawling script was run over a dark web hosted e-commerce platform, collecting records of products being sold on the site. The purpose of this notebook is to analyse the data collected.
## 1. Introduction
 
### Database schema:

In [6]:
db.pandas_query('SHOW tables;')

,Tables_in_dark_web
0,tblCategory
1,tblProduct
2,tblSubCategory
3,tblVendor
4,tblWebsite


<hr />
##### Data definition can be obtained by the following query:

In [7]:
db.pandas_query('DESCRIBE tblProduct;')

,Field,Type,Null,Key,Default,Extra
0,product_id,int(20),NO,PRI,None,auto_increment
1,product_model,varchar(200),NO,UNI,None,
2,product_name,varchar(100),NO,,None,
3,product_description,text,NO,,None,
4,product_origin,varchar(50),NO,,None,
5,product_postage,varchar(50),NO,,None,
6,product_price,varchar(50),NO,,None,
7,vendor_id,int(10),NO,MUL,None,
8,subCategory_id,int(10),NO,MUL,None,
9,time_stamp,date,NO,,None,


<hr>
## 2. Descriptive Analytics

We will start off by describing the dataset in terms of the products table and its properties:

### 2.1 Product Categories

The site divides the product listing into a number of categories and subcategories. We will now have a look at those categories and see which products are the most popular in this site.

In [24]:
dataframe = db.pandas_query('''
        SELECT B.subcategory_name, count(A.product_name) as 'Total Products' 
        FROM tblProduct A
        LEFT JOIN tblSubCategory B
            ON A.subCategory_id = B.subCategory_id
        GROUP BY B.subcategory_name
        ORDER BY -count(A.product_name)
        LIMIT 20;
        ''')
dataframe.plot()

ImportError: No module named 'matplotlib'

In [9]:
db.get_product_bitcoin_prices().fetchall()

((1, '0.36154597 BTC', '2015-07-22'),
 (2, '0.14425684 BTC', '2015-07-22'),
 (3, '0.09020571 BTC', '2015-07-22'),
 (4, '0.36154597 BTC', '2015-07-22'),
 (5, '0.35793051 BTC', '2015-07-22'),
 (6, '0.19523482 BTC', '2015-07-22'),
 (7, '0.11533316 BTC', '2015-07-22'),
 (8, '0.19523482 BTC', '2015-07-22'),
 (9, '0.18055605 BTC', '2015-07-22'),
 (10, '0.19497400 BTC', '2015-07-22'))

In [10]:
db.insert_converted_prices('GBP', '2015-01-01', '2015-12-30')

### Pulling bitcoin price indexes in GBP ###
#### Updating Records ###


True